<a href="https://colab.research.google.com/github/IreneDeNevi/Banks/blob/main/banks_project_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL Project: Top 10 Largest Banks by Market Capitalization

## Project Overview
This notebook extracts data about the top 10 largest banks in the world by market capitalization, transforms the data by converting USD to GBP, EUR, and INR, and loads the data to both CSV and SQLite database.

## Setup Instructions
1. Run all cells in order
2. The script will automatically download the exchange rate CSV
3. All outputs (CSV, database, log file) will be created in your Colab environment
4. You can download the files from the Files panel on the left

## Step 1: Install Required Libraries
Most libraries are pre-installed in Colab, but let's ensure everything is available.

In [1]:
# Install required libraries (if not already installed)
!pip install beautifulsoup4 requests pandas numpy -q

## Step 2: Import Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

print("All libraries imported successfully!")

All libraries imported successfully!


## Step 3: Define Configuration Parameters

In [3]:
# Configuration Parameters
URL = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
EXCHANGE_RATE_CSV = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
OUTPUT_CSV_PATH = './Largest_banks_data.csv'
DATABASE_NAME = 'Banks.db'
TABLE_NAME = 'Largest_banks'
LOG_FILE = 'code_log.txt'

print("Configuration complete!")

Configuration complete!


## Task 1: Logging Function

In [4]:
def log_progress(message):
    """
    Log the progress of the code execution with timestamp.

    Parameters:
    message (str): The log message to be written
    """
    timestamp_format = '%Y-%m-%d %H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)

    with open(LOG_FILE, 'a') as f:
        f.write(f'{timestamp} : {message}\n')

    print(f'{timestamp} : {message}')

# Clear previous log file
open(LOG_FILE, 'w').close()
log_progress('Preliminaries complete. Initiating ETL process')

2025-11-17 15:10:13 : Preliminaries complete. Initiating ETL process


## Task 2: Extract Function

In [5]:
def extract(url, table_attribs):
    """
    Extract tabular data from the given URL.

    Parameters:
    url (str): The URL to extract data from
    table_attribs (list): List of column names for the dataframe

    Returns:
    pd.DataFrame: Extracted data as a pandas DataFrame
    """
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all tables in the page
        tables = soup.find_all('table', {'class': 'wikitable'})

        # Initialize empty dataframe
        df = pd.DataFrame(columns=table_attribs)

        # Find the correct table (by market capitalization)
        if tables:
            table = tables[0]
            rows = table.find_all('tr')

            # Extract data from rows (skip header)
            for row in rows[1:]:
                cols = row.find_all('td')
                if len(cols) >= 3:  # Ensure row has enough columns
                    # Extract bank name (second column)
                    bank_name = cols[1].get_text(strip=True)

                    # Extract market cap (third column)
                    market_cap = cols[2].get_text(strip=True)

                    # Clean market cap value
                    market_cap = market_cap.replace(',', '').replace('\n', '')

                    # Try to convert to float
                    try:
                        market_cap_value = float(market_cap)

                        # Add to dataframe
                        new_row = pd.DataFrame({
                            table_attribs[0]: [bank_name],
                            table_attribs[1]: [market_cap_value]
                        })
                        df = pd.concat([df, new_row], ignore_index=True)

                        # Stop after getting top 10 banks
                        if len(df) >= 10:
                            break
                    except ValueError:
                        continue

        return df

    except Exception as e:
        log_progress(f'Error during extraction: {str(e)}')
        return pd.DataFrame(columns=table_attribs)

# Execute extraction
log_progress('Data extraction started')
extracted_data = extract(URL, ['Name', 'MC_USD_Billion'])
log_progress('Data extraction complete. Initiating Transformation process')

print("\nExtracted Data:")
print(extracted_data)

2025-11-17 15:10:13 : Data extraction started
2025-11-17 15:10:14 : Data extraction complete. Initiating Transformation process

Extracted Data:
                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                        HSBC Holdings PLC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


/tmp/ipython-input-2084099673.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


## Task 3: Transform Function

In [6]:
def transform(df, csv_path):
    """
    Transform the dataframe by adding currency conversion columns.

    Parameters:
    df (pd.DataFrame): Input dataframe with bank data
    csv_path (str): Path to the exchange rate CSV file

    Returns:
    pd.DataFrame: Transformed dataframe with additional currency columns
    """
    try:
        # Read exchange rate CSV
        exchange_rates = pd.read_csv(csv_path)

        print("\nExchange Rates:")
        print(exchange_rates)

        # Create a dictionary for easy lookup
        exchange_dict = exchange_rates.set_index('Currency')['Rate'].to_dict()

        # Add new columns with currency conversions
        df['MC_GBP_Billion'] = [np.round(x * exchange_dict['GBP'], 2) for x in df['MC_USD_Billion']]
        df['MC_EUR_Billion'] = [np.round(x * exchange_dict['EUR'], 2) for x in df['MC_USD_Billion']]
        df['MC_INR_Billion'] = [np.round(x * exchange_dict['INR'], 2) for x in df['MC_USD_Billion']]

        return df

    except Exception as e:
        log_progress(f'Error during transformation: {str(e)}')
        return df

# Execute transformation
log_progress('Data transformation started')
transformed_data = transform(extracted_data, EXCHANGE_RATE_CSV)
log_progress('Data transformation complete. Initiating Loading process')

print("\nTransformed Data:")
print(transformed_data)

2025-11-17 15:10:14 : Data transformation started

Exchange Rates:
  Currency   Rate
0      EUR   0.93
1      GBP   0.80
2      INR  82.95
2025-11-17 15:10:15 : Data transformation complete. Initiating Loading process

Transformed Data:
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82   

## Task 4: Load to CSV Function

In [7]:
def load_to_csv(df, output_path):
    """
    Load the dataframe to a CSV file.

    Parameters:
    df (pd.DataFrame): Dataframe to save
    output_path (str): Path where CSV will be saved
    """
    try:
        df.to_csv(output_path, index=False)
        log_progress(f'Data saved to CSV file: {output_path}')
    except Exception as e:
        log_progress(f'Error saving to CSV: {str(e)}')

# Execute CSV loading
log_progress('Loading data to CSV')
load_to_csv(transformed_data, OUTPUT_CSV_PATH)
log_progress('Data saved to CSV file')

# Verify CSV was created
print("\nVerifying CSV file:")
csv_check = pd.read_csv(OUTPUT_CSV_PATH)
print(csv_check.head())

2025-11-17 15:10:15 : Loading data to CSV
2025-11-17 15:10:15 : Data saved to CSV file: ./Largest_banks_data.csv
2025-11-17 15:10:15 : Data saved to CSV file

Verifying CSV file:
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75  
3          149.43        13328.41  
4          146.86        13098.63  


## Task 5: Load to Database Function

In [8]:
def load_to_db(df, sql_connection, table_name):
    """
    Load the dataframe to an SQL database table.

    Parameters:
    df (pd.DataFrame): Dataframe to save
    sql_connection: SQLite database connection
    table_name (str): Name of the table to create/replace
    """
    try:
        df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
        log_progress(f'Data loaded to database table: {table_name}')
    except Exception as e:
        log_progress(f'Error loading to database: {str(e)}')

# Execute database loading
log_progress('Initiating connection to SQLite database')
conn = sqlite3.connect(DATABASE_NAME)
log_progress('SQL Connection initiated')

load_to_db(transformed_data, conn, TABLE_NAME)
log_progress('Data loaded to Database as a table, Executing queries')

2025-11-17 15:10:15 : Initiating connection to SQLite database
2025-11-17 15:10:15 : SQL Connection initiated
2025-11-17 15:10:15 : Data loaded to database table: Largest_banks
2025-11-17 15:10:15 : Data loaded to Database as a table, Executing queries


## Task 6: Run Database Queries

In [9]:
def run_query(query, sql_connection):
    """
    Run a query on the database and print results.

    Parameters:
    query (str): SQL query to execute
    sql_connection: SQLite database connection
    """
    try:
        print(f"\nQuery: {query}")
        result = pd.read_sql_query(query, sql_connection)
        print(result)
        log_progress(f'Query executed: {query}')
    except Exception as e:
        log_progress(f'Error running query: {str(e)}')
        print(f"Error: {e}")

print("\n" + "="*50)
print("Running Database Queries")
print("="*50)

# Query 1: Print all contents of the table
query1 = f"SELECT * FROM {TABLE_NAME}"
run_query(query1, conn)

# Query 2: Print average market capitalization in GBP
query2 = f"SELECT AVG(MC_GBP_Billion) AS Average_MC_GBP FROM {TABLE_NAME}"
run_query(query2, conn)

# Query 3: Print only the names of top 5 banks
query3 = f"SELECT Name FROM {TABLE_NAME} LIMIT 5"
run_query(query3, conn)

log_progress('Process Complete')

# Close database connection
conn.close()
log_progress('Server Connection closed')


Running Database Queries

Query: SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1      

## Task 7: Verify Log Entries

In [10]:
print("\n" + "="*50)
print("Log File Contents")
print("="*50)

with open(LOG_FILE, 'r') as f:
    log_contents = f.read()
    print(log_contents)

print("\n" + "="*50)
print("ETL Process Completed Successfully!")
print("="*50)
print("\nGenerated Files:")
print(f"1. CSV File: {OUTPUT_CSV_PATH}")
print(f"2. Database: {DATABASE_NAME}")
print(f"3. Log File: {LOG_FILE}")
print("\nYou can download these files from the Files panel on the left.")


Log File Contents
2025-11-17 15:10:13 : Preliminaries complete. Initiating ETL process
2025-11-17 15:10:13 : Data extraction started
2025-11-17 15:10:14 : Data extraction complete. Initiating Transformation process
2025-11-17 15:10:14 : Data transformation started
2025-11-17 15:10:15 : Data transformation complete. Initiating Loading process
2025-11-17 15:10:15 : Loading data to CSV
2025-11-17 15:10:15 : Data saved to CSV file: ./Largest_banks_data.csv
2025-11-17 15:10:15 : Data saved to CSV file
2025-11-17 15:10:15 : Initiating connection to SQLite database
2025-11-17 15:10:15 : SQL Connection initiated
2025-11-17 15:10:15 : Data loaded to database table: Largest_banks
2025-11-17 15:10:15 : Data loaded to Database as a table, Executing queries
2025-11-17 15:10:15 : Query executed: SELECT * FROM Largest_banks
2025-11-17 15:10:15 : Query executed: SELECT AVG(MC_GBP_Billion) AS Average_MC_GBP FROM Largest_banks
2025-11-17 15:10:15 : Query executed: SELECT Name FROM Largest_banks LIMIT 5

## Download Files (Optional)
Run this cell to download all generated files to your local machine.

In [11]:
from google.colab import files

# Download CSV file
files.download(OUTPUT_CSV_PATH)

# Download database file
files.download(DATABASE_NAME)

# Download log file
files.download(LOG_FILE)

print("All files downloaded!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All files downloaded!
